In [99]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [101]:
x=linspace(0,5,10000)

In [81]:
def V0(x,D,re,a):
    V0=D*(1-exp(-a*(x-re)))**2
    return V0

In [82]:
D=0.15
re=0.64
a=6.5

In [83]:
def VHO(x,k,re,E0):
    VHO=0.5*k*(x-re)**2+E0
    return VHO

In [84]:
ka=6
rea=0.55
E0a=0.3
kb=40
reb=1
E0b=0.55

In [85]:
gamma=0.75

In [86]:
def V1(x,ka,rea,E0a,kb,reb,E0b,gamma):
    VHOa=0.5*ka*(x-rea)**2+E0a
    VHOb=0.5*kb*(x-reb)**2+E0b
    V1=0.5*(VHOa+VHOb)-0.5*sqrt((VHOa-VHOb)**2+2*gamma**2)
    return V1

In [87]:
plot(x,V0(x,D,re,a),label='Morse Potential')
plot(x,V1(x,ka,rea,E0a,kb,reb,E0b,gamma), label='Excited State Potential')
xlim(0.2,1.5)
ylim(0,0.5)
xlabel('$x/a_0$')
ylabel('$V/E_h$')
title('Ground and First Excited State Potentials')
legend(loc='upper right')
show()

In [9]:
from qdyn import propagator, animate_dynamics

In [10]:
from numpy.polynomial.hermite import hermval

def eigen_ho(x,v,m,k,r):
    """Calculates the eigenfunction of the harmonic oscillator system.
    
    Arguments
    x: is a space coordinate.
    v: is the vibrational quantum number.
    m: is the mas of the system.
    k: is the force constant of the harmonic potential.
    """
    
    hermite_sum=zeros(v+1)
    hermite_sum[-1]=1
    return 1/(2**v * math.factorial(v))**0.5 * (((m*k)**0.5)/pi)**0.25 * np.e**(-(x-r)**2 * ((m*k)**0.5)/2) * hermval((m*k)**0.25 * (x-r),hermite_sum) 

Ground State Simulation

In [12]:
x=linspace(0,10,100000)
dt=10
x0=0
v=0
m=14583
k=2*D*a**2
r=0.64
psi0=eigen_ho(x,x0,m,k,r)

# Add the initial wavefunction to the array with the wavefunction evolution
wf_dynamics=array([psi0])

In [40]:
for steps in range(100): # How many time steps/how many times to apply the propagator
    psi=propagator(x,wf_dynamics[-1],m,dt,V0,D,re,a) # Apply the propagator to the last element in the array
    wf_dynamics=append(wf_dynamics,[psi],axis=0) # Add the new wavefunction to the array
animate_dynamics(x,wf_dynamics,dt,V0(x,D,re,a), xlim(0,1.5),V_ylim=(0,0.3),psi_ylim=(-0.35,5))

In [11]:
from numpy import trapz

In [12]:
def auto(prop_wf):
    auto=conj(prop_wf[0])*prop_wf
    return trapz(auto,x)

In [79]:
t=linspace(0,dt*steps,101)
plot(t,auto(wf_dynamics))

C:\Users\sahil\Anaconda3\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [12]:
import pickle

In [ ]:
pickle.dump(wf_dynamics, open('gs data','wb'))

In [13]:
from numpy.fft import fft,ifft,fftshift,ifftshift,fftfreq

In [80]:
t_gs=linspace(0,dt*steps,len(auto(wf_dynamics)))
omega_gs=ifftshift(ifft(auto(wf_dynamics),norm='ortho'))
freq_gs=fftshift(fftfreq(len(t_gs),d=dt))

Ground State Power Spectrum

In [81]:
plot(freq_gs,abs(omega_gs))

Excited State Simulation

In [14]:
x=linspace(0,5,10000)
dt=10
x0=0
v=0
m=14583
k=2*D*a**2
r=0.64
psi0=eigen_ho(x,x0,m,k,r)

# Add the initial wavefunction to the array with the wavefunction evolution
wf_dynamics1=array([psi0])

for steps in range(1000): # How many time steps/how many times to apply the propagator
    psi=propagator(x,wf_dynamics1[-1],m,dt,V1,ka,rea,E0a,kb,reb,E0b,gamma) # Apply the propagator to the last element in the array
    wf_dynamics1=append(wf_dynamics1,[psi],axis=0) # Add the new wavefunction to the array
animate_dynamics(x,wf_dynamics1,dt,V1(x,ka,rea,E0a,kb,reb,E0b,gamma), xlim(0,1.5),V_ylim=(0.15,0.3),psi_ylim=(-0.35,5))

In [65]:
excited=auto(wf_dynamics1)
t=linspace(0,dt*steps,steps+2)
plot(t,excited)

C:\Users\sahil\Anaconda3\lib\site-packages\numpy\core\numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [153]:
pickle.dump(wf_dynamics1, open('excited data2','wb'))

In [16]:
t_excited=linspace(0,dt*1000,len(excited))
omega_excited=ifftshift(ifft(excited,norm='ortho'))
freq_excited=2*pi*fftshift(fftfreq(len(t_excited),d=dt))

Excited State Power Spectrum

In [17]:
plot(freq_excited,abs(omega_excited))

In [18]:
E0=0.5*sqrt(k/m)
f=[]

In [19]:
s=auto(wf_dynamics1)
t

array([   0.  ,    9.99,   19.98, ..., 9970.02, 9980.01, 9990.  ])

In [20]:
f=s*exp(1j*E0*t)
f

array([ 1.        +0.j        , -0.60106296-0.79442194j,
       -0.27026609+0.94756127j, ..., -0.38182993-0.71501595j,
       -0.37624459+0.73517733j,  0.83423955-0.1180039j ])

In [21]:
f1=abs(ifftshift(ifft(f,norm="ortho")))

In [22]:
f2=f1/(137*3)*(2*pi)**1.5*freq_excited

correct for ortho fourier transform which multiplies by 1/sqrt(2pi)

Excited State Vibronic Spectrum

In [118]:
plot(freq_excited,abs(f2)/max(f2))
xlabel('Angular Frequency/radau$^{-1}$')
ylabel('Intensity (normalised)')
title('Vibronic Spectrum')

Text(0.5, 1.0, 'Vibronic Spectrum')

In [24]:
from scipy.signal import find_peaks

In [25]:
E0_0=[]
peak_index0=find_peaks(f2, height=0.001)[0]
for i in peak_index0:
    E0_0=E0_0 +[freq_excited[i]]
E01=[]
for i in E0_0:
    E01.append(i+E0)
E01

[0.23003874099259464,
 0.2344325768717412,
 0.2538909929079617,
 0.26456173718588905,
 0.27711555398345067,
 0.2896693707810123,
 0.302850878418452]

In [26]:
AC0e=[]
for i in range(len(E01)):
    AC0e.append(exp(E01[i]*1j*t)*transpose(wf_dynamics1))
integral=trapz(AC0e,t)
integral

array([[ 2.35590348e-08+3.03285925e-07j, -8.70727897e-07-3.31048688e-07j,
        -2.62120808e-07+2.40205350e-07j, ...,
        -1.46538029e-07-1.02945528e-06j, -1.96258663e-07-1.99182985e-07j,
        -1.97392121e-07-1.06015050e-06j],
       [-2.37337219e-07-3.81992554e-07j,  2.53918558e-07+1.09151062e-07j,
         6.46884754e-07+6.72666813e-07j, ...,
        -9.46983298e-07+1.19471318e-07j,  3.90015556e-07-5.34059109e-07j,
        -1.91920149e-07-5.83979585e-07j],
       [-1.08983001e-08-1.48539077e-07j, -8.61588939e-07-1.32242391e-06j,
        -1.65471477e-07-3.07652382e-07j, ...,
        -1.62751318e-07+1.67375925e-08j,  2.68538048e-07-4.82567954e-07j,
        -1.74197879e-08-7.19370682e-07j],
       ...,
       [-4.77764433e-08-4.08173394e-07j, -1.22719058e-07-1.54844999e-07j,
         2.75568110e-07-2.52840846e-07j, ...,
        -4.50655763e-07+3.81459044e-08j, -2.93392879e-08-2.75655212e-08j,
        -3.96464284e-07-1.73865507e-07j],
       [ 2.87655939e-06+2.34081838e-05j, -9.

In [27]:
for i in range(len(integral)):
    plot(x,abs(integral[i]))

Excited State Eigenstates

In [102]:
eigenstates=integral/(2*pi)
for i in range(len(eigenstates)):
    plot(x,abs(eigenstates[i]),label='$|\Lambda^{}|$'.format(i,i))
xlabel('$x/a_0$')
ylabel('$|\Lambda^{i}|$')
legend(loc='upper right')

In [29]:
c_values=sqrt(trapz(conj(eigenstates)*eigenstates))
c_values

array([11056.97715345+0.j, 58665.33106467+0.j,  6851.79745595+0.j,
       10736.76798823+0.j,   397.76926295+0.j,   900.91002887+0.j,
         263.07097638+0.j])

Normalised Eigenstates

In [66]:
fig=plt.figure()
#create axes for potential plot
potplot=plt.gca()
potplot.plot(x,V1(x,ka,rea,E0a,kb,reb,E0b,gamma),color="gray")
potplot.set(xlabel="$x/a_0$",ylabel="$V/E_h$",ylim=(0.1,0.4))
Lamplot=potplot.twinx()
norm_eigenstates=[]
for state in range(len(c_values)):
    for i in eigenstates[state]:
        norm_eigenstates.append(i/c_values[state])
    graphical=0.05*state+abs(array(norm_eigenstates))
    plot(x,graphical,label='$|\Lambda^{}|$'.format(state,state))
    norm_eigenstates=[]
Lamplot.set(ylabel='$|\Lambda$|', ylim=(-0.05,0.35))
xlim(0,1.5)
legend(loc='upper right')

Dynamics

In [56]:
V1list=V1(x,ka,rea,E0a,kb,reb,E0b,gamma).tolist()
plot(V1(x,ka,rea,E0a,kb,reb,E0b,gamma))
ylim(0,0.3)

(0, 0.3)

In [57]:
def find(v,x):
    for i in range(len(x)):
        if x[i] > v:
            return i

In [58]:
find(0.55,x)

1100

In [105]:
V1max=max(V1list[1100:2000])
split=V1list.index(V1max)

In [106]:
wf_CC=conj(wf_dynamics1[:1500])*(wf_dynamics1[:1500])

In [122]:
pop1=[]
pop2=[]


for i in range(len(wf_CC)):
    totI=(trapz(abs(wf_CC[i])))
    pop1+=[(trapz(abs(wf_CC[i][:split])))/totI]
    pop2+=[(trapz(abs(wf_CC[i][split:])))/totI]
pop1avg=mean(pop1[200:])
pop2avg=mean(pop2[200:])
plot(t,pop1, label='Higher Well Population')
plot(t,pop2, label='Lower Well Population')
title('Well Populations')
xlabel('Time/au')
ylabel('Population fraction')
legend(loc='center right')
axhline(pop1avg,linestyle='-')
axhline(pop2avg, linestyle='-')
for i in range(len(pop1)):
    if pop1[i]<pop1avg:
        tte=i
        break

In [111]:
tte*dt

130